<div style="margin: 0 auto 10px; height: 70px; border: 2px solid gray; border-radius: 6px;">
  <div style="float: left; margin: 5px 10px 5px 10px; "><img src="img/bfh.jpg" /></div>
  <div style="float: right; margin: 20px 30px 0; font-size: 15pt; font-weight: bold; color: #98b7d2;"><a href="https://moodle.bfh.ch/course/view.php?id=39255" style="color: #98b7d2;">BTE5476 - Project-Oriented Digital Signal Processing </a></div>
</div>
<div style="clear: both; font-size: 30pt; font-weight: bold; color: #64788b; margin-left: 30px;">
    Fractional Resampling
</div>

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sp
from scipy.io import wavfile
import IPython

In [ ]:
plt.rcParams["figure.figsize"] = (9,2.5)

# Multirate signal processing

Consider a discrete-time audio signal $x_i[n]$ whose nominal sampling rate is $F_i$ Hz; if we want to play back the signal using an interpolator whose rate is $F_o \neq F_i$ we need to process the signal if we want to avoid very audible artefacts:

In [ ]:
Fi_audio, x_audio = wavfile.read('data/oao.wav')
IPython.display.Audio(x_audio, rate=Fi_audio)

In [ ]:
IPython.display.Audio(x_audio, rate=int(Fi_audio * 0.6))

In [ ]:
IPython.display.Audio(x_audio, rate=int(Fi_audio * 1.5))

In this notebook we will consider the problem of changing the implicit sampling rate of a signal entirely in the discrete-time domain. We will use the following symbols throughout:

 * $x_i[n]$ is the **input** sequence, with a sampling rate of $F_i$ samples per second. The sampling period is $T_i = 1/F_i$ seconds 
 * the **output** sequence is $x_o[n]$ and we want its sampling rate to be $F_o$ (with $T_o = 1/F_o$)
 * the rate change factor is $$\alpha = \frac{F_o}{F_i}$$
 * if $\alpha > 1$ the signal will be **upsampled**
 * if $\alpha < 1$ the signal will be **downsampled**

# Upsampling

An **upsampler** increases the implicit sampling rate of a signal by an *integer* factor $N$. This is accomplished by appending $N-1$ zeros to every input sample (interpolation) and by filtering the result using a lowpass with cutoff $\omega_c = \pi /N$. The filtering operation is needed to remove the high-frequency artefacts introduced by the extra zero samples. Note that upsampling is always invertible (no information is lost).


## Downsampling
An **downsampler** reduced the implicit sampling rate of a signal by an *integer* factor $M$. This is accomplished by first filtering the signal using a lowpass with cutoff $\omega_c = \pi /M$ and then discarding $M-1$ samples out of every $M$ samples (decimation). The filtering operation is needed to avoid aliasing; because of this, and contrary to upsampling, in general downsampling entails a loss of information and cannot be inverted.


## Rational sampling rate changes

<img width="600" style="float:right;" src="img/multirate.png">

When the target sampling rate is a rational multiple of the input rate, the rate change can be implemented by cascading an upsampler and a downsampler. Assume $F_o/F_i = N/M$:
 * since upsampling is "safe" we start with an interpolator by $N$ and a lowpass with cutoff $\pi/N$
 * we cascade a lowpass with cutoff $\pi/M$ and decimate by $M$
 * the cascade of the two lowpass filters can be replaced by a single lowpass with cutoff $\omega_c = \min\{\pi/N, \pi/M\}$

## Exercise

Implement the upsampling and downsampling operations as described above

In [ ]:
def upsample(x: np.ndarray, N: int) -> np.ndarray:
    y = np.zeros(len(x) * N)
    ...
    return y


def downsample(x: np.ndarray, M: int) -> np.ndarray:
    y = np.zeros(len(x) // M)
    ...
    return y

In [ ]:
N = 3
IPython.display.Audio(upsample(x_audio, N), rate=int(Fi_audio * N))

In [ ]:
IPython.display.Audio(downsample(x_audio, N), rate=int(Fi_audio / N))

# Fractional resampling

The theoretical approach to rational rate changes becomes difficult to implement when the $F_o/F_i$ cannot be reduced to a ratio of small integers. For instance, to go from 44.1 kHz to 48 kHz (i.e. from CD audio to DVD audio), we have
$$
    \frac{48000}{44100} = \frac{160}{147}
$$
which requires an intermediate signal (the signal that goes through the lowpass filter) at more than 6 MHz! That's a lot of computations per sample. 

To get around this problem, we use fractional resampling instead. 

## Resampling rate

Given $F_i$ and $F_o$, a fractional resampler must generates $N_o$ output samples for every $N_i$ input samples where

$$
    \frac{N_o}{N_i} = \frac{F_o}{F_i} = \alpha.
$$

For maximum efficiency, we want $N_i, N_o$ to be coprime. The following simple function uses Euclid's algorithm to reduce a ratio into lowest terms:

In [ ]:
def simplify(A: int, B: int) -> (int, int):
    a, b = A, B
    while a != b:
        if a > b:
            a = a - b
        else:
            b = b - a
    return A // a, B // b

We can test the function on the usual CD to DVD sampling rate change and verify the 160/147 value:

In [ ]:
print(simplify(48000, 44100))

## Resampling strategy

A practical fractional resampler with rate change $\alpha$ computes each output sample as a *local* interpolation of a finite number of input samples.

Assume $p(t; x, n)$ is a polynomial or piecewise-polynomial interpolator such that
$$
    p(k; x, n) = x[n-k], \qquad k = 0, \pm 1, \ldots, \pm P, \qquad  P \in \mathbb{N};
$$

the resampler works as follows:

 * the "clock" of the system is the output rate $F_o$
 * for each output index $m$, find the input index closest in time, $n[m] = \mbox{round}(m / \alpha)$. This is called the "anchor" index for $m$
 * call $\tau[m]$ the time offset between the $m$-th output sample and its anchor $$
   \tau[m] = m \, T_o - n[m]\, T_i;
   $$ we will show later that $|\tau[m]| \le T_i/2$, that is, the offset is at most one half of the input sampling period (in magnitude)
 * fit the interpolation function over $2P+1$ samples centered on the anchor $x_i[n[m]]$
 * compute the output sample as $x_o[m] = p(\tau[m]; x_i, n[m])$.

## Local interpolation functions

For symmetry reasons, we will consider interpolation functions that use an odd number of samples around the anchor. 

### Zero-order interpolator
In this case ($P=0$) we simply return the anchor point for all values of the time offset 
$$
    p_0(t; x, n) = x[n]
$$

### First-order interpolator
In this case ($P=1$) we use the value of the piecewise-linear interpolation between the anchor and each neighboring sample
$$
    p_1(t; x, n) = \begin{cases}
        x[n]\,(1-t) + x[n+1]\,t & t \ge 0 \\
        -x[n-1]\,t + x[n]\,(1+t) & t < 0 
        \end{cases}
$$

<div style="margin: 10px;"><img src="img/linint.png" width="600"></div>

### Second-order interpolator
In this case ($P=1$) we use a quadratic interpolation of the anchor and its two neighboring samples
$$
    p_2(t; x, n) = x[n-1]\,\frac{t(t-1)}{2} + x[n]\, (1-t^2) + x[n+1]\,\frac{t(t+1)}{2}
$$

<div style="margin: 10px;"><img src="img/quadint.png" width="600"></div>


### Higher-order interpolators
Piecewise-polynomial interpolators of any order can be defined as well, for instance using Lagrange polynomials. An interpolator of order $L$ will use $2P+1$ points around the anchor with $P=\lceil L/2 \rceil$. In general, howver, a quadratic interpolator is enough.

## Implementation

In [ ]:
def p0(t: float, x: np.ndarray, n: int) -> float:
    return x[n]

def p1(t: float, x: np.ndarray, n: int) -> float:
    if t > 0:
        return x[n-1] * (1-t) + x[n] * t
    else:
        return -x[n-2] * t + x[n-1] * (1+t)

def p2(t: float, x: np.ndarray, n: int) -> float:
    return x[n-2] * t * (t - 1) / 2 + x[n-1] * (1 - t) * (1 + t) + x[n] * t * (t + 1) / 2


def resample(x: np.ndarray, output_rate: int, input_rate: int, interp=p0) -> np.ndarray:
    No, Ni = simplify(output_rate, input_rate)
    y = np.zeros(No * len(x) // Ni) 
    for m in range(0, len(y)):
        n = int(m * Ni / No + 0.5) 
        tau = (m * Ni / No) - n
        y[m] = interp(tau, x, n)
    return y

In [ ]:
x = np.cos(2 * np.pi * 440 / 44100 * np.arange(0, 44100))
IPython.display.Audio(x, rate=44100)

In [ ]:
y = resample(x, 48000, 44100, p2)
IPython.display.Audio(y, rate=48000)

# Efficient implementation

If the rate change factor is fixed and known, we can improve the effciency of the resampling algorithm if we notice that:
 1. piecewise-polynomial interpolation can be implemented as a convolution with an FIR of length $2P+1$
 2. the coefficients of the FIR change for each output sample as a function of $\tau[m]$
 3. the sequence $\tau[m]$ has only a finite number of different values so the number of FIRs is finite and the coefficients can be precomputed

## Interpolation as convolution

As an example, consider $p_2(t; x, n)$:

$$
\begin{align}
  p_2(t; x, n) &= x[n-1]\,\frac{t(t-1)}{2} + x[n]\, (1-t^2) + x[n+1]\,\frac{t(t+1)}{2}= x[n-1]\,\frac{t(t-1)}{2} + x[n]\, (1-t^2) + x[n+1]\,\frac{t(t+1)}{2} \\
    &= \sum_{k=-1}^{1} x[n-k] h_t[k] \\
    &= (x \ast h_t)[n]
\end{align}
$$
where
$$
    h_t[k] = \begin{cases}
        t(t+1)/2 & k = -1 \\
        1-t^2 & k = 0 \\
        t(t-1)/2 & k = 1 \\
        0 & \mbox{otherwise}
        \end{cases}
$$

## Repeating anchor patterns

For a rate change $\alpha = N_o/N_i$, the values of $\tau[m]$ form a periodic sequence with period $N_o$. Indeed, the anchor points are determined like so:

 * the output sample $x_o[m]$ occurs at time $m \, T_o$
 * the associated anchor index is $n[m] = \mbox{round}(m / \alpha)$, which occurs at time $n[m] \, /T_i$
 * we have $$
       \begin{align*}
       \tau[m] &= m \, T_o -  n[m] \, /T_i\\
            &= m \, T_o - \mbox{round}(m / \alpha) \, T_i \\ 
            &= T_i(m / \alpha - \mbox{round}(m / \alpha))
        \end{align*}
    $$
    so that $|\tau| \le T_i/2$. 

Since $\alpha = N_o/N_i$, then $\tau(kN_o) = 0$: input and output samples will align exactly every $N_o$ output samples (or, equivalently, every $N_i$ input samples) and the sequence $\tau[m]$ will repeat.

This can be easily understood graphically: in the figures below, the top line shows the times for the input samples $x_i[n]$, with dotted red lines separating intervals of width $1/F_i$. The bottom line shows the times of the output samples $x_o[n]$ and the blue arrows show the corresponding input anchor sample.

### Downsampling:
Let's first consider downsampling, with a ratio $N_o/N_i = 4/5$; the operation generates 4 output samples for every 5 input samples; the five possible values for $\tau$ are $0, 0.25, 0.5, -0.25$. Since the output rate is smaller than the input rate, not all input sampler become anchor points and some are skipped.

![title](img/down.png)



### Upsampling
In the following figure, the signal is upsampled with a ratio $N_o/N_i = 8/5$ generates 8 output samples for every 5 input samples; the 8 possible values for $\tau$ are $0, -0.375, 0.25, -0.125, -0.5, 0.125, -0.25, 0.375$. Since the output rate is larger than the input rate, some input samples will act as the anchor for more than one output sample.

![title](img/up.png)



## Exercise 

Implement the fractional resampler using precomputed FIR interpolators

In [ ]:
def resample_fir(x: np.ndarray, output_rate: int, input_rate: int, order=1) -> np.ndarray:
    No, Ni = simplify(output_rate, input_rate)
    y = np.zeros(No * len(x) // Ni) 
    
    # your code here
    
    return y

We can now test the function on a simple sinusoid; we generate the sinusoid at 44.1 KHz and the pitch should not change when we resample:

In [ ]:
Fi = 44100
x = np.cos(2 * np.pi * 440 / Fi * np.arange(0, Fi))
IPython.display.Audio(x, rate=Fi)

In [ ]:
Fo = 48000
IPython.display.Audio(resample_fir(x, Fo, Fi), rate=Fo)

In [ ]:
Fo = 16000
IPython.display.Audio(resample_fir(x, Fo, Fi), rate=Fo)

We can now test the resampler on an audio file; can you hear a difference between first and second order interpolators?

In [ ]:
Fo = 48000
IPython.display.Audio(resample_fir(x_audio, Fo, Fi_audio, order=1), rate=Fo)

In [ ]:
Fo = 48000
IPython.display.Audio(resample_fir(x_audio, Fo, Fi_audio, order=2), rate=Fo)

Note how aliasing begins to appear when we downsample too much; this is because local interpolation does not provide a good lowpass response

In [ ]:
Fo = 8000
IPython.display.Audio(resample_fir(x_audio, Fo, Fi_audio), rate=Fo)